<h2>Installing required packages</h2>

In [26]:
#!pip install tensorflow

In [27]:
#!pip install keras

In [28]:
#!pip install matplotlib

In [29]:
#!pip install pennylane --upgrade

<h2>Necessary imports</h2>

In [30]:
from keras.models import Sequential
from keras.layers import Dense

In [31]:
from sklearn import metrics

In [64]:
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean

In [32]:
import json
from Filters import *
from Data_load import data_load
import matplotlib.pyplot as plt

import keras

from utils import *

import time

## Import the images
data = data_load()
train_images, train_labels, test_images, test_labels = data.data_mnist()

6000


In [33]:
print("shape of train images", train_images[0].shape)
print("shape of test images", test_images[0].shape)

shape of train images (28, 28, 1)
shape of test images (28, 28, 1)


In [34]:
print("length of train images", len(train_images))
print("length of test images", len(test_images))

length of train images 6000
length of test images 1000


<h2>Using 28x28 images</h2>

In [35]:
# This function returns the modified dataset
# It is a script to get a dataset of 2 labels only
def trainTestData(train_images, train_labels, test_images, test_labels, label1, label2, n_train, n_test):
    n_train_images, n_train_labels, n_test_images, n_test_labels = [], [], [], []
    i = 0
    while (n_train > 0 and i < len(train_images)):
        if train_labels[i] == label1:
            n_train_images.append(train_images[i])
            n_train_labels.append(label1)
            n_train -= 1
        if train_labels[i] == label2:
            n_train_images.append(train_images[i])
            n_train_labels.append(label2)
            n_train -= 1
        i = i+1
    i = 0
    while(n_test>0 and i < len(test_images)):
        if test_labels[i] == label1:
            n_test_images.append(test_images[i])
            n_test_labels.append(label1)
            n_test -= 1
        if test_labels[i] == label2:
            n_test_images.append(test_images[i])
            n_test_labels.append(label2)
            n_test -= 1
        i = i+1
    return n_train_images, n_train_labels, n_test_images, n_test_labels

In [36]:
# getting a dataset of numbers 3 and 5 with train dataset of 80 and test dataset of 35
train_images, train_labels, test_images, test_labels = trainTestData(train_images, train_labels, test_images, test_labels, 3, 5, 500, 60)

In [37]:
# this functions makes the train dataset balanced
# number of images for number 3 same as that for number 5
def balanceDataSet(BAS, train_labels):
    new_train_images_3 = []
    new_train_images_5 = []

    new_train_labels_3 = []
    new_train_labels_5 = []

    for i in range(len(BAS)):
      if train_labels[i] == 3:
        new_train_images_3.append(BAS[i])
        new_train_labels_3.append(3)
      else:
        new_train_images_5.append(BAS[i])
        new_train_labels_5.append(5)
    return new_train_images_3[0:150] + new_train_images_5[0:150], new_train_labels_3[0:150] + new_train_labels_5[0:150]

In [38]:
train_images, train_labels = balanceDataSet(train_images, train_labels)
# to have a fair comparison of performance
org_train_images, org_train_labels, org_test_images, org_test_labels = train_images, train_labels, test_images, test_labels

In [39]:
print("length of train images", len(train_images))
print("length of test images", len(test_images))

length of train images 300
length of test images 60


In [40]:
train_images = np.array(train_images)
test_images = np.array(test_images)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
train_images.shape

(300, 28, 28, 1)

In [41]:
model = Sequential()
# 784 is 28*28
# 10 is the number of neurons
model.add(Dense(10, activation='relu', input_shape=(784,)))
model.add(Dense(10, activation='relu'))
# sigmoid activation used for binary classification
model.add(Dense(2, activation='sigmoid'))

In [42]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [43]:
# changing data shape from (28, 28) t0 (784, )
train_data = train_images.reshape((train_images.shape[0], 784))

#encoding train labels using one-hot encoding
categories = np.array([3, 5])
encoded_train_labels = np.zeros((len(train_labels), len(categories)))
for i in range(len(train_labels)):
  j = np.where(categories == train_labels[i])
  encoded_train_labels[i, j] = 1

In [44]:
model.reset_states()
model.fit(train_data, encoded_train_labels, validation_split=0.2, epochs=10)

Epoch 1/10
8/8 [==============================] - 3s 110ms/step - loss: 0.7898 - accuracy: 0.4625 - val_loss: 0.8761 - val_accuracy: 0.0167
Epoch 2/10
8/8 [==============================] - 0s 15ms/step - loss: 0.6250 - accuracy: 0.6292 - val_loss: 1.0053 - val_accuracy: 0.0000e+00
Epoch 3/10
8/8 [==============================] - 0s 14ms/step - loss: 0.5903 - accuracy: 0.6250 - val_loss: 1.0189 - val_accuracy: 0.0000e+00
Epoch 4/10
8/8 [==============================] - 0s 12ms/step - loss: 0.5542 - accuracy: 0.6458 - val_loss: 0.9214 - val_accuracy: 0.1500
Epoch 5/10
8/8 [==============================] - 0s 13ms/step - loss: 0.5055 - accuracy: 0.7083 - val_loss: 0.7759 - val_accuracy: 0.4667
Epoch 6/10
8/8 [==============================] - 0s 12ms/step - loss: 0.4125 - accuracy: 0.8542 - val_loss: 0.4694 - val_accuracy: 0.7500
Epoch 7/10
8/8 [==============================] - 0s 15ms/step - loss: 0.3083 - accuracy: 0.9000 - val_loss: 0.3367 - val_accuracy: 0.8667
Epoch 8/10
8/8 [==

In [45]:
# changing data shape from (28, 28) t0 (784, )
test_data = test_images.reshape((len(test_labels), 784))

# encoding test labels using one-hot encoding
categories = np.array([3, 5])
encoded_test_labels = np.zeros((len(test_labels), len(categories)))
for i in range(len(test_labels)):
  j = np.where(categories == test_labels[i])
  encoded_test_labels[i, j] = 1

In [46]:
model.evaluate(test_data, encoded_test_labels)

2/2 [==============================] - 0s 8ms/step - loss: 0.2264 - accuracy: 0.9667


[0.22640666365623474, 0.9666666388511658]

In [47]:
predicted = model.predict(test_data)
predicted

2/2 [==============================] - 0s 6ms/step


array([[0.08539055, 0.88550586],
       [0.42206246, 0.5822893 ],
       [0.59990835, 0.41219598],
       [0.15061715, 0.8092798 ],
       [0.78151774, 0.20862004],
       [0.6632408 , 0.35185534],
       [0.65457183, 0.35365245],
       [0.41837674, 0.56707585],
       [0.8078884 , 0.22008955],
       [0.09017101, 0.8599103 ],
       [0.40914732, 0.5509962 ],
       [0.22139734, 0.75329995],
       [0.80936545, 0.22546417],
       [0.90223753, 0.16032763],
       [0.76407915, 0.28533226],
       [0.5285718 , 0.49312383],
       [0.8712327 , 0.20151803],
       [0.8103461 , 0.21586336],
       [0.2061235 , 0.73284334],
       [0.74087214, 0.30829427],
       [0.44563416, 0.51041794],
       [0.05144162, 0.9207523 ],
       [0.22163574, 0.71682286],
       [0.05554933, 0.90909314],
       [0.846076  , 0.24176505],
       [0.06311683, 0.9097609 ],
       [0.2520113 , 0.7024108 ],
       [0.01720934, 0.96363294],
       [0.83538854, 0.2351112 ],
       [0.04686346, 0.9098712 ],
       [0.

<h2>Using 4x4 images (quantum filter)</h2>

In [48]:
train_images, train_labels, test_images, test_labels = org_train_images, org_train_labels, org_test_images, org_test_labels

In [49]:
# Applying the quantum filter for the first time
# (28, 28) to (14, 14) image
start_time = time.time()

filtered_train_images = apply_filter(train_images, 6, data.n_channels, data.n_layers, True)
filtered_test_images = apply_filter(test_images, 6, data.n_channels, data.n_layers, True)

end_time = time.time()

time_taken = end_time - start_time

print(time_taken/60, " minutes")

fitlered pre-processing of images, with type = 6
fitlered pre-processing of images, with type = 6
6.358977731068929  minutes


In [50]:
train_images = filtered_train_images
test_images = filtered_test_images

In [51]:
print("shape of train images", train_images[0].shape)
print("shape of test images", test_images[0].shape)

shape of train images (14, 14, 1)
shape of test images (14, 14, 1)


In [52]:
# Applying the quantum filter for the second time
# (14, 14) to (7,7) image
start_time = time.time()

filtered_train_images = apply_filter(train_images, 6, data.n_channels, data.n_layers, True)
filtered_test_images = apply_filter(test_images, 6, data.n_channels, data.n_layers, True)

end_time = time.time()

time_taken = end_time - start_time

print(time_taken/60, " minutes")

fitlered pre-processing of images, with type = 6
fitlered pre-processing of images, with type = 6
1.5404990474383036  minutes


In [53]:
train_images = filtered_train_images
test_images = filtered_test_images

In [54]:
print("shape of train images", train_images[0].shape)
print("shape of test images", test_images[0].shape)

shape of train images (7, 7, 1)
shape of test images (7, 7, 1)


In [55]:
# function to perform max pooling using keras
def conv(train_images, test_images):

    new_train_images, new_test_images = [], []

    for img in train_images:
        img = np.reshape(img, [1, 7, 7, 1])
        max_pool_2d = keras.layers.MaxPooling2D(pool_size=(1, 1), strides=(2, 2), padding="valid")
        new_train_images.append(np.reshape(max_pool_2d(img), [4, 4]))

    for img in test_images:
        img = np.reshape(img, [1, 7, 7, 1])
        max_pool_2d = keras.layers.MaxPooling2D(pool_size=(1, 1), strides=(2, 2), padding="valid")
        new_test_images.append(np.reshape(max_pool_2d(img), [4, 4]))
    return new_train_images, new_test_images

In [56]:
train_images, test_images = conv(train_images, test_images)

In [57]:
train_images = np.array(train_images)
test_images = np.array(test_images)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
train_images.shape

(300, 4, 4)

In [58]:
# 16 used instead of 784 since 4*4 = 16
train_data = train_images.reshape((len(train_images), 16))

categories = np.array([3, 5])
encoded_train_labels = np.zeros((len(train_labels), len(categories)))
for i in range(len(train_labels)):
  j = np.where(categories == train_labels[i])
  encoded_train_labels[i, j] = 1

In [59]:
model2 = Sequential()
model2.add(Dense(10, activation='relu', input_shape=(16,)))
model2.add(Dense(10, activation='relu'))
model2.add(Dense(2, activation='sigmoid'))

In [60]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [61]:
model2.reset_states()
model2.fit(train_data, encoded_train_labels, validation_split=0.2, epochs=10)

Epoch 1/10
8/8 [==============================] - 1s 39ms/step - loss: 0.6554 - accuracy: 0.6250 - val_loss: 0.8985 - val_accuracy: 0.0167
Epoch 2/10
8/8 [==============================] - 0s 8ms/step - loss: 0.6514 - accuracy: 0.6292 - val_loss: 0.9106 - val_accuracy: 0.0500
Epoch 3/10
8/8 [==============================] - 0s 8ms/step - loss: 0.6485 - accuracy: 0.6292 - val_loss: 0.9084 - val_accuracy: 0.0833
Epoch 4/10
8/8 [==============================] - 0s 7ms/step - loss: 0.6453 - accuracy: 0.6375 - val_loss: 0.9159 - val_accuracy: 0.0833
Epoch 5/10
8/8 [==============================] - 0s 8ms/step - loss: 0.6432 - accuracy: 0.6417 - val_loss: 0.9316 - val_accuracy: 0.0833
Epoch 6/10
8/8 [==============================] - 0s 7ms/step - loss: 0.6401 - accuracy: 0.6417 - val_loss: 0.9115 - val_accuracy: 0.0833
Epoch 7/10
8/8 [==============================] - 0s 8ms/step - loss: 0.6373 - accuracy: 0.6542 - val_loss: 0.9049 - val_accuracy: 0.1000
Epoch 8/10
8/8 [=================

In [62]:
# 16 used instead of 784 since 4*4 = 16
test_data = test_images.reshape((len(test_labels), 16))

categories = np.array([3, 5])
encoded_test_labels = np.zeros((len(test_labels), len(categories)))
for i in range(len(test_labels)):
  j = np.where(categories == test_labels[i])
  encoded_test_labels[i, j] = 1

In [63]:
model2.evaluate(test_data, encoded_test_labels)

2/2 [==============================] - 0s 8ms/step - loss: 0.7178 - accuracy: 0.5333


[0.7178212404251099, 0.5333333611488342]

<h2>Using classical filter</h2>

In [68]:
train_images, train_labels, test_images, test_labels = org_train_images, org_train_labels, org_test_images, org_test_labels

In [69]:
# changing the shape and size of the train images and storing in BAS
BAS = []
for img in train_images:
    # using sickit image to rescale to (4,4)
    img = rescale(img, 1/7, anti_aliasing=False)
    img = img.reshape(4,4)
    BAS.append(img)

# changing the shape and size of the test images and storing in BAS_test
BAS_test = []
for img in test_images:
    # using sickit image to rescale to (4,4)
    img = rescale(img, 1/7, anti_aliasing=False)
    img = img.reshape(4,4)
    BAS_test.append(img)

train_images = BAS[:]
test_images = BAS_test[:]

In [70]:
train_images = np.array(train_images)
test_images = np.array(test_images)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
train_images.shape

(300, 4, 4)

In [71]:
# 16 used instead of 784 since 4*4 = 16
train_data = train_images.reshape((len(train_images), 16))

categories = np.array([3, 5])
encoded_train_labels = np.zeros((len(train_labels), len(categories)))
for i in range(len(train_labels)):
  j = np.where(categories == train_labels[i])
  encoded_train_labels[i, j] = 1

# 16 used instead of 784 since 4*4 = 16
test_data = test_images.reshape((len(test_labels), 16))

categories = np.array([3, 5])
encoded_test_labels = np.zeros((len(test_labels), len(categories)))
for i in range(len(test_labels)):
  j = np.where(categories == test_labels[i])
  encoded_test_labels[i, j] = 1

In [72]:
model3 = Sequential()
model3.add(Dense(10, activation='relu', input_shape=(16,)))
model3.add(Dense(10, activation='relu'))
model3.add(Dense(2, activation='sigmoid'))

In [73]:
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [74]:
model3.reset_states()
model3.fit(train_data, encoded_train_labels, validation_split=0.2, epochs=10)

Epoch 1/10
8/8 [==============================] - 1s 40ms/step - loss: 0.6751 - accuracy: 0.6208 - val_loss: 0.7464 - val_accuracy: 0.0167
Epoch 2/10
8/8 [==============================] - 0s 9ms/step - loss: 0.6659 - accuracy: 0.6208 - val_loss: 0.7643 - val_accuracy: 0.0000e+00
Epoch 3/10
8/8 [==============================] - 0s 10ms/step - loss: 0.6570 - accuracy: 0.6208 - val_loss: 0.7838 - val_accuracy: 0.0000e+00
Epoch 4/10
8/8 [==============================] - 0s 12ms/step - loss: 0.6469 - accuracy: 0.6208 - val_loss: 0.8013 - val_accuracy: 0.0000e+00
Epoch 5/10
8/8 [==============================] - 0s 11ms/step - loss: 0.6375 - accuracy: 0.6250 - val_loss: 0.8145 - val_accuracy: 0.0167
Epoch 6/10
8/8 [==============================] - 0s 11ms/step - loss: 0.6306 - accuracy: 0.6375 - val_loss: 0.8352 - val_accuracy: 0.0167
Epoch 7/10
8/8 [==============================] - 0s 10ms/step - loss: 0.6211 - accuracy: 0.6417 - val_loss: 0.8458 - val_accuracy: 0.0333
Epoch 8/10
8/8 [

In [75]:
model3.evaluate(test_data, encoded_test_labels)

2/2 [==============================] - 0s 9ms/step - loss: 0.6788 - accuracy: 0.5667


[0.6787984371185303, 0.5666666626930237]